In [28]:
# import numpy as np
# import matplotlib.pyplot as plt

# project = 'newfault'
# traindate = '2024-10-01'
# testdate = '2024-10-02'
# traindate_path = '/home/sdybing/gnss-picker/cnn_models_outputs/' + project + '_fq_train/models/traindate_' + traindate + '/'
# test_outputs_path = traindate_path + 'data/'
# figure_save_dir = traindate_path + 'figures/'

# fqtest_data = np.load(test_outputs_path + testdate + '_fqtest_orig_data.npy')
# fqtest_metadata = np.load(test_outputs_path + testdate + '_fqtest_metadata.npy')
# fqtest_target = np.load(test_outputs_path + testdate + '_fqtest_target.npy')
# fqtest_predictions = np.load(test_outputs_path + testdate + '_fqtest_predictions.npy')

# results = np.load(test_outputs_path + 'fakequakes_testing/fqtest_metadata_withresults.npy')

# num_fqtest = len(fqtest_predictions)
# best_thresh = 0.135 # From code 2


In [30]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from obspy.geodetics import gps2dist_azimuth

path = '/home/sdybing/gnss-picker/cnn_models_outputs/newfault_fq_train/models/traindate_2024-10-01/data/'

data = np.load(path + '2024-10-16_gfast_bigger_fqtest_orig_data.npy')
metadata = np.load(path + 'bigger_pgd_test_metadata_with_hypdists.npy')
targets = np.load(path + '2024-10-16_gfast_bigger_fqtest_target.npy')
predictions = np.load(path + '2024-10-16_gfast_bigger_fqtest_predictions.npy')

# data = np.load(path + '2024-10-16_gfast_bigger_fqtest_orig_data.npy')
# metadata = np.load(path + 'bigger_pgd_test_metadata_with_hypdists.npy')

# Columns:

# 0: FQ rupture name
# 1: station name
# 2: magnitude
# 3: P-arrival index
# 4: FQ rupture hypocenter lat
# 5: FQ rupture hypocenter lon
# 6: FQ rupture hypocenter depth (km)
# 7: station lat
# 8: station lon
# 9: hypocentral distance (m)

In [31]:
print(metadata[0])
print(data.shape)

['newfault.001022' 'IMPS' '6.2624' '41' '35.692601' '-117.545871' '0.8'
 '34.157563' '-115.1451' '277688.3029341335']
(20683, 128, 3)


In [32]:
# Calculating PGDs from the waveforms - HAVE TO USE ORIGINAL, NOT NORM, DATA

pgds = []

for i in range(len(data)):
    n_data = data[i,:,0]
    e_data = data[i,:,1]
    z_data = data[i,:,2]
    pgd = np.max(np.sqrt((n_data)**2+(e_data)**2+(z_data)**2))
    pgds.append(pgd)

print(np.array(pgds).shape)
np.save(path + 'gfast_bigger_test_metadata_pgds.npy', np.array(pgds))

(20683,)


In [33]:
# Calculating SNRs from the waveforms using the target Gaussian peak as the arrival time

targets_count = []

SNRs_N = []
SNRs_E = []
SNRs_Z = []
mean_SNRs = []

for idx in range(len(targets)):
    
    target_min = min(targets[idx,:])
    target_max = max(targets[idx,:])
    target_range = target_max - target_min
    
    if target_range != 0:
        target_max_idx = np.argmax(targets[idx,:])
        targets_count.append(target_max_idx)
    
        p_arrival_index = int(target_max_idx) # The index in the sample that the P-wave arrives at
        if p_arrival_index != float(metadata[idx,3]):
            print(idx)
        
        '''
        In this section, I calculate the signal-to-noise ratio of the data. I 
        aim to use a window of 20 seconds before the P-wave arrival time as the 
        noise, and a window of 20 seconds after the P-wave arrival time as the 
        signal. I take the standard deviation of these segments and divide 
        signal/noise (or after/before) to get the SNR.
        
        Sometimes the P-wave arrival time is too close to the start or end of the
        sample, and this causes issues. I've added conditions for these cases.
        '''
        
        preeq_std_end = p_arrival_index # The end of the 20 second 'noise' section before the earthquake is the P-wave arrival index
        
        if preeq_std_end <= 10: # Ask Diego if this is reasonable # Try 10
        
            # If P-wave pick is at zero - can't calculate a pre-eq standard deviation. 
            # OR the P-wave pick is too close to zero, it throws off the SNR values by a LOT.
            
            SNR_N = 'nan' # Just skip it (at least 10 cases for Z component with weird SNRs - one over 10,000!)
            SNR_E = 'nan'
            SNR_Z = 'nan'
        
        elif preeq_std_end > 10 and preeq_std_end <= 20: # If the pre-earthquake noise window is smaller than 20 seconds...
            
            preeq_std_start = 0
            
            posteq_std_start = p_arrival_index # Start the section for the "signal" at the P-wave arrival index
            posteq_std_end = posteq_std_start + 20
            # posteq_std_end = posteq_std_start + p_arrival_index # If the window before is less than 20 because the arrival time is less than 20, this makes the window after that same length
            
            std_before_N = np.std(data[idx,preeq_std_start:preeq_std_end,0]) # Take the standard deviation of the sections for each component
            std_after_N = np.std(data[idx,posteq_std_start:posteq_std_end,0])
            std_before_E = np.std(data[idx,preeq_std_start:preeq_std_end,1])
            std_after_E = np.std(data[idx,posteq_std_start:posteq_std_end,1])
            std_before_Z = np.std(data[idx,preeq_std_start:preeq_std_end,2])
            std_after_Z = np.std(data[idx,posteq_std_start:posteq_std_end,2])
            
            if std_before_N == 0 or std_before_E == 0 or std_before_Z == 0: # If any of the denominators are zeros, we get 'inf' in the results
                
                SNR_N = 'nan' # Skip 'em
                SNR_E = 'nan'
                SNR_Z = 'nan'
                
            else: # Calculate the SNR
                
                SNR_N = std_after_N / std_before_N
                SNR_E = std_after_E / std_before_E
                SNR_Z = std_after_Z / std_before_Z
        
        elif preeq_std_end > 20 and preeq_std_end <= 108: # Standard case where the P-wave arrival is nicely in the middle somewhere
            
            preeq_std_start = preeq_std_end - 20
            
            posteq_std_start = p_arrival_index
            posteq_std_end = posteq_std_start + 20
        
            std_before_N = np.std(data[idx,preeq_std_start:preeq_std_end,0]) # Take the standard deviation of the sections for each component
            std_after_N = np.std(data[idx,posteq_std_start:posteq_std_end,0])
            std_before_E = np.std(data[idx,preeq_std_start:preeq_std_end,1])
            std_after_E = np.std(data[idx,posteq_std_start:posteq_std_end,1])
            std_before_Z = np.std(data[idx,preeq_std_start:preeq_std_end,2])
            std_after_Z = np.std(data[idx,posteq_std_start:posteq_std_end,2])
            
            if std_before_N == 0 or std_before_E == 0 or std_before_Z == 0:
                
                SNR_N = 'nan'
                SNR_E = 'nan'
                SNR_Z = 'nan'
                
            else:
                
                SNR_N = std_after_N / std_before_N
                SNR_E = std_after_E / std_before_E
                SNR_Z = std_after_Z / std_before_Z
            
        elif preeq_std_end > 108 and preeq_std_end < 128: # End edge case - the "signal" period is less than 20 seconds long
            
            preeq_std_start = preeq_std_end - 20
            
            posteq_std_start = p_arrival_index # Should the below be 127 instead??
            posteq_std_end = posteq_std_start + (128 - p_arrival_index) # Make the signal period end at the end of the sample at 128 to avoid errors
        
            std_before_N = np.std(data[idx,preeq_std_start:preeq_std_end,0]) # Take the standard deviation of the sections for each component
            std_after_N = np.std(data[idx,posteq_std_start:posteq_std_end,0])
            std_before_E = np.std(data[idx,preeq_std_start:preeq_std_end,1])
            std_after_E = np.std(data[idx,posteq_std_start:posteq_std_end,1])
            std_before_Z = np.std(data[idx,preeq_std_start:preeq_std_end,2])
            std_after_Z = np.std(data[idx,posteq_std_start:posteq_std_end,2])
            
            if std_before_N == 0 or std_before_E == 0 or std_before_Z == 0:
                
                SNR_N = 'nan'
                SNR_E = 'nan'
                SNR_Z = 'nan'
                
            else:
                
                SNR_N = std_after_N / std_before_N
                SNR_E = std_after_E / std_before_E
                SNR_Z = std_after_Z / std_before_Z
            
        else: # Covers if the pick is exactly at 128, the end of the sample.
            
            # Can't get a post-eq std because the earthquake arrives at the end of the sample
            
            SNR_N = 'nan' # Skip 'em (5 cases)
            SNR_E = 'nan'
            SNR_Z = 'nan'
            
        '''
        Add the calculated SNRs (or 'nan's for issues) to the lists.
        '''
        
        # if SNR_N == 0:
            
        #     print(idx)
            
        SNRs_N.append(SNR_N)
        SNRs_E.append(SNR_E)
        SNRs_Z.append(SNR_Z)
        
    elif target_range == 0:
        
        SNRs_N.append('nan')
        SNRs_E.append('nan')
        SNRs_Z.append('nan')
        
    try:
        mean_SNR = (SNR_N + SNR_E + SNR_Z) / 3
    except:
        mean_SNR = 'nan'
        
    mean_SNRs.append(mean_SNR)

print(len(SNRs_N))
print(len(SNRs_E))
print(len(SNRs_Z))
print(len(mean_SNRs))


20683
20683
20683
20683


In [34]:
np.save(path + 'gfast_bigger_test_SNRs_N.npy', np.array(SNRs_N))
np.save(path + 'gfast_bigger_test_SNRs_E.npy', np.array(SNRs_E))
np.save(path + 'gfast_bigger_test_SNRs_Z.npy', np.array(SNRs_Z))
np.save(path + 'gfast_bigger_test_mean_SNRs.npy', np.array(mean_SNRs))

In [35]:
# Plots to check and make sure stuff looks right

testmags = []
for ii in range(len(metadata)):
    mag = metadata[ii,2]
    if mag == 'nan':
        testmags.append(np.nan)
    else:
        testmags.append(float(mag))
        
testsnrs = []
for iii in range(len(SNRs_N)):
    snr = SNRs_N[iii]
    if snr == 'nan':
        testsnrs.append(np.nan)
    else:
        testsnrs.append(float(snr))
        
testpgds = []
for iiii in range(len(pgds)):
    pgd = pgds[iiii]
    if pgd == 'nan':
        testpgds.append(np.nan)
    else:
        testpgds.append(float(pgd))

logsnrs = np.log10(testsnrs)
logpgds = np.log10(testpgds)

# Remove NaNs to be able to make this quick plot
fixmags = []
numremovedmags = []
for iv in range(len(testmags)):
    mag = testmags[iv]
    if np.isnan(mag):
        numremovedmags.append(1)
    else:
        fixmags.append(mag)    
print('Number of NaN mags removed: ' + str(len(numremovedmags)))
print('Number of good mags left: ' + str(len(fixmags)))

fixpgds = []
numremovedpgds = []
for iv in range(len(logpgds)):
    pgd = logpgds[iv]
    if np.isnan(pgd):
        numremovedpgds.append(1)
    else:
        fixpgds.append(pgd)    
print('Number of NaN PGDs removed: ' + str(len(numremovedpgds)))
print('Number of good PGDs left: ' + str(len(fixpgds)))

fixsnrs = []
numremovedsnrs = []
for iv in range(len(logsnrs)):
    snr = logsnrs[iv]
    if np.isnan(snr):
        numremovedsnrs.append(1)
    elif np.isinf(snr):
        numremovedsnrs.append(1)
    else:
        fixsnrs.append(snr) 
print('Number of NaN SNRs removed: ' + str(len(numremovedsnrs)))
print('Number of good SNRs left: ' + str(len(fixsnrs)))

Number of NaN mags removed: 0
Number of good mags left: 20683
Number of NaN PGDs removed: 0
Number of good PGDs left: 20683
Number of NaN SNRs removed: 738
Number of good SNRs left: 19945


In [36]:
# # Make a list of indices for rows in the testing dataset that actually have earthquakes
# # and for rows that the CNN correctly found those earthquakes

# rows_w_eqs = []
# correct_eq_inds = []

# for idx in range(len(results)):
    
#     if results[idx,0] == 'nan':
#         pass
#     else:
#         rows_w_eqs.append(idx)
    
#     if results[idx,3] == 'true pos':
#         correct_eq_inds.append(idx)

# print(len(rows_w_eqs))
# print(len(correct_eq_inds))

# np.save(test_outputs_path + 'fakequakes_testing/fqtest_data_rows_w_eqs.npy', np.array(rows_w_eqs))
# np.save(test_outputs_path + 'fakequakes_testing/fqtest_data_rows_w_truepos_result.npy', np.array(correct_eq_inds))

In [37]:
# Add the PGDs and SNRs to the results metadata array

pgds_vector = np.array(pgds).reshape(len(pgds),1) 
a = np.append(metadata, pgds_vector, axis = 1) 
# Now contains rupture name, station name, magnitude, result, and PGD

SNR_N_vector = np.array(SNRs_N).reshape(len(SNRs_N),1)
SNR_E_vector = np.array(SNRs_E).reshape(len(SNRs_E),1)
SNR_Z_vector = np.array(SNRs_Z).reshape(len(SNRs_Z),1)
mean_SNR_vector = np.array(mean_SNRs).reshape(len(mean_SNRs),1)

b = np.append(a, SNR_N_vector, axis = 1)
c = np.append(b, SNR_E_vector, axis = 1)
d = np.append(c, SNR_Z_vector, axis = 1)
new_meta_array = np.append(d, mean_SNR_vector, axis = 1)

print(new_meta_array.shape)

# Columns:

# 0: FQ rupture name
# 1: station name
# 2: magnitude
# 3: P-arrival index
# 4: FQ rupture hypocenter lat
# 5: FQ rupture hypocenter lon
# 6: FQ rupture hypocenter depth (km)
# 7: station lat
# 8: station lon
# 9: hypocentral distance (m)
# 10. PGD (m)
# 11. SNRN
# 12. SNRE
# 13. SNRZ
# 14. mean SNR

np.save(path + 'pgd_bigger_test_metadata_w_hypdist_pgd_snrs.npy', new_meta_array)

(20683, 15)


In [38]:
new_meta_array.shape

(20683, 15)

In [39]:
print(fqtest_metadata[0])
print(results[0])
print(new_meta_array[0])

['newfault.001132' 'NHRG' '6.3333']
['newfault.001132' 'NHRG' '6.3333' 'false neg']
['newfault.001132' 'NHRG' '6.3333' 'false neg' '0.021478579700444114'
 '0.9797122685583282' '0.9337101853976125' '1.4218615090358038']


In [40]:
np.mean(new_meta_array[rows_w_eqs,4].astype(float))

0.03591847188120218

In [41]:
all_eq_mags = np.asfarray(new_meta_array[rows_w_eqs, 2]) # The magnitudes of all of the earthquakes
correct_eq_mags = np.asfarray(new_meta_array[correct_eq_inds, 2]) # The magnitudes of all the earthquakes the CNN found

all_eq_avg_mag = np.mean(all_eq_mags)
correct_eq_avg_mag = np.mean(correct_eq_mags)

print('Average magnitude of all FakeQuakes: ' + str(round((all_eq_avg_mag),2)))
print('Average magnitude of FakeQuakes the CNN correctly found: ' + str(round((correct_eq_avg_mag),2)))

Average magnitude of all FakeQuakes: 5.83
Average magnitude of FakeQuakes the CNN correctly found: 6.89


In [42]:
all_eq_pgds = np.asfarray(new_meta_array[rows_w_eqs, 4]) # The PGDs of all of the earthquakes
correct_eq_pgds = np.asfarray(new_meta_array[correct_eq_inds, 4]) # The PGDs of all the earthquakes the CNN found

all_eq_avg_PGD = np.mean(all_eq_pgds)
correct_eq_avg_PGD = np.mean(correct_eq_pgds)

print('Average PGD of all FakeQuakes: ' + str(round((all_eq_avg_PGD * 100),2)) + ' cm')
print('Average PGD of FakeQuakes the CNN correctly found: ' + str(round((correct_eq_avg_PGD * 100),2)) + ' cm')

Average PGD of all FakeQuakes: 3.59 cm
Average PGD of FakeQuakes the CNN correctly found: 5.69 cm


In [43]:
# Because of the edge cases in the loop above, there are some nans in our SNR vectors.
# We can't calculate averages with nans, so we need to find the rows with nans and
# just remove them for the sake of this calculation.

h = np.where(new_meta_array[rows_w_eqs,5] == 'nan') # Finds nans in SNRN column for all earthquakes
non_nan_rows_w_eqs = np.delete(rows_w_eqs, h) # Removes those rows

j = np.where(new_meta_array[correct_eq_inds,5] == 'nan') # Finds nans in SNRN for the earthquakes the CNN found
non_nan_correct_eq_inds = np.delete(correct_eq_inds, j) # Removes those rows

In [44]:
# Now I just grab the good SNRs out of the new metadata array and calculate the averages.

all_eq_SNR_N = np.asfarray(new_meta_array[non_nan_rows_w_eqs, 5])
all_eq_SNR_E = np.asfarray(new_meta_array[non_nan_rows_w_eqs, 6])
all_eq_SNR_Z = np.asfarray(new_meta_array[non_nan_rows_w_eqs, 7])

correct_eq_SNR_N = np.asfarray(new_meta_array[non_nan_correct_eq_inds, 5])
correct_eq_SNR_E = np.asfarray(new_meta_array[non_nan_correct_eq_inds, 6])
correct_eq_SNR_Z = np.asfarray(new_meta_array[non_nan_correct_eq_inds, 7])

all_eq_SNR_N_avg = np.mean(all_eq_SNR_N)
all_eq_SNR_E_avg = np.mean(all_eq_SNR_E)
all_eq_SNR_Z_avg = np.mean(all_eq_SNR_Z)

correct_eq_SNR_N_avg = np.mean(correct_eq_SNR_N)
correct_eq_SNR_E_avg = np.mean(correct_eq_SNR_E)
correct_eq_SNR_Z_avg = np.mean(correct_eq_SNR_Z)

# print(len(all_eq_SNR_N))
# print(len(correct_eq_SNR_N))

print('Average N-S component SNR of all FakeQuakes: ' + str(round(all_eq_SNR_N_avg,2)))
print('Average N-S component SNR of FakeQuakes the CNN correctly found: ' + str(round(correct_eq_SNR_N_avg,2)))
print('-------------------------------------------------------------------')

print('Average E-W component SNR of all FakeQuakes: ' + str(round(all_eq_SNR_E_avg,2)))
print('Average E-W component SNR of FakeQuakes the CNN correctly found: ' + str(round(correct_eq_SNR_E_avg,2)))
print('-------------------------------------------------------------------')

print('Average Z component SNR of all earthquakes: ' + str(round(all_eq_SNR_Z_avg,2)))
print('Average Z component SNR of earthquakes the CNN correctly found: ' + str(round(correct_eq_SNR_Z_avg,2)))

Average N-S component SNR of all FakeQuakes: 1.38
Average N-S component SNR of FakeQuakes the CNN correctly found: 2.57
-------------------------------------------------------------------
Average E-W component SNR of all FakeQuakes: 1.51
Average E-W component SNR of FakeQuakes the CNN correctly found: 3.12
-------------------------------------------------------------------
Average Z component SNR of all earthquakes: 1.07
Average Z component SNR of earthquakes the CNN correctly found: 1.2
